In [17]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display

# Load the dataset
recipes_df = pd.read_pickle('Yummly Clean.pkl')

# Assuming column names are 'cuisine', 'rating', and 'ingredients'. Adjust as necessary based on your DataFrame inspection
dish_name_col = 'recipeName'
ingredients_col = 'ingredients'  # Adjust if your ingredients column name is different
cuisine_col = 'cuisine'  # Adjust if your cuisine column name is different
rating_col = 'rating'  # Adjust if your rating column name is different

# Calculate the average rating per cuisine
cuisine_ratings = recipes_df.groupby(cuisine_col)[rating_col].agg(['mean']).reset_index()
cuisine_ratings.columns = ['Cuisine', 'Average Rating']

# Calculate top ingredients for each cuisine
top_ingredients_by_cuisine = (
    recipes_df.explode(ingredients_col)
    .groupby(cuisine_col)[ingredients_col]
    .value_counts()
    .groupby(level=0, group_keys=False)
    .head(5)
    .reset_index(name='count')
    .pivot(index=cuisine_col, columns=ingredients_col, values='count')
)

# Dropdown widget for selecting cuisine
cuisine_dropdown = widgets.Dropdown(
    options=cuisine_ratings['Cuisine'].unique().tolist(),
    description='Select Cuisine:',
    disabled=False
)

# Output widgets for displaying the average rating and top ingredients
rating_output = widgets.Output()
top_ingredients_output = widgets.Output()
random_recipes_output = widgets.Output()

# Function to update outputs based on selected cuisine
def on_cuisine_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        cuisine = change['new']

        # Display the average rating
        avg_rating = cuisine_ratings[cuisine_ratings['Cuisine'] == cuisine]['Average Rating'].values[0]
        with rating_output:
            rating_output.clear_output()
            print(f"\nAverage Rating for {cuisine}: {avg_rating:.2f}")

        # Display the top ingredients
        if cuisine in top_ingredients_by_cuisine.index:
            top_ingredients = top_ingredients_by_cuisine.loc[cuisine].nlargest(5).index.tolist()
            with top_ingredients_output:
                top_ingredients_output.clear_output()
                print(f"\nTop ingredients for {cuisine} cuisine:")
                print(', '.join(top_ingredients))
        else:
            with top_ingredients_output:
                top_ingredients_output.clear_output()
                print(f"No data available for {cuisine} cuisine.")

        # Display 10 random recipes from the selected cuisine
        random_recipes = recipes_df[recipes_df[cuisine_col] == cuisine][dish_name_col].sample(10).tolist()
        with random_recipes_output:
          random_recipes_output.clear_output()
          print(f"\nCommon recipes for {cuisine} cuisine:")
          for recipe in random_recipes:
            print(recipe)


# Observe changes in the dropdown to update outputs accordingly
cuisine_dropdown.observe(on_cuisine_change, names='value')

# Display widgets
display(cuisine_dropdown, rating_output, top_ingredients_output, random_recipes_output)


Dropdown(description='Select Cuisine:', options=('American', 'Asian', 'Barbecue', 'Cajun & Creole', 'Chinese',…

Output()

Output()

Output()